# Load libraries

In [10]:
import pandas as pd
import numpy as np

In [11]:
from matplotlib import*
import matplotlib.pyplot as plt
from matplotlib.cm import register_cmap

import seaborn as sns

from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool

from pprint import pprint

In [12]:
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy import stats


In [13]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
from sklearn.preprocessing import normalize
from sklearn.preprocessing import Normalizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA

# Read data from spreadsheet

In [18]:
raw_input = pd.read_csv('data/cleaned.csv', index_col='County')

In [40]:
raw_input

,pct_rur_10,pop_10,pop_00,pop_90,pop_80,pop_70,hsg_10,hsg_00,hsg_90,hsg_80,...,cp_id_fm_02,cp_id_acr_02,cp_fl_fm_02,cp_fl_acr_02,cp_sm_fm_02,cp_sm_acr_02,wd_fm_02,wd_acr_02,wd_p1_fm_02,wd_p1_acr_02
County,,,,,,,,,,,,,,,,,,,,,
anderson,34.7,75129,71330,68250,67346,60300,34717,32451,29323,25849,...,72.0,2364.0,14.0,93.0,3.0,16.0,369.0,12974.0,192.0,3790.0
bedford,55.6,45058,37586,30411,27916,25039,18360,14990,12638,10814,...,191.0,5373.0,37.0,412.0,1.0,12.0,859.0,38296.0,610.0,22449.0
benton,78.5,16489,16537,14524,14901,12126,8975,8595,7107,6526,...,167.0,6895.0,13.0,66.0,2.0,6.0,346.0,19621.0,175.0,5589.0
bledsoe,100.0,12876,12367,9669,9478,7643,5718,5142,3771,3406,...,84.0,4771.0,11.0,189.0,6.0,6.0,318.0,21127.0,182.0,6505.0
blount,32.6,123010,105823,85969,77770,63744,55266,47059,36532,30836,...,143.0,2234.0,28.0,2231.0,2.0,50.0,712.0,17460.0,424.0,8212.0
bradley,33.0,98963,87965,73712,67547,50686,41395,36820,29562,24705,...,80.0,2229.0,13.0,156.0,6.0,30.0,473.0,23121.0,283.0,7998.0
campbell,55.0,40716,39854,35079,34923,26045,19966,18527,14817,13250,...,63.0,698.0,22.0,393.0,3.0,145.0,245.0,8297.0,160.0,4235.0
cannon,81.1,13801,12826,10467,10234,8467,6037,5420,4368,4002,...,87.0,2438.0,26.0,346.0,2.0,15.0,567.0,30387.0,387.0,17462.0
carroll,83.1,28522,29473,27514,28285,25741,13184,13055,11783,11306,...,330.0,14790.0,19.0,439.0,11.0,70.0,576.0,36829.0,288.0,8978.0


In [48]:
metrics = list(raw_input.columns)
counties = list(raw_input.index)
print(len(metrics), len(counties))

188 95


# Step for principal component analysis

In [22]:
# Create scaler: scaler
scaler = StandardScaler()

# Fit the pipeline to 'samples'
scaler_df = pd.DataFrame(scaler.fit_transform(raw_input))

In [52]:
scaler_df.columns = metrics
scaler_df.index = counties

scaler_df

In [25]:
# Covariance matrix
cov_df = pd.DataFrame(np.cov(scaler_df.T))

In [56]:
cov_df.index = metrics
cov_df.columns = metrics
cov_df

,pct_rur_10,pop_10,pop_00,pop_90,pop_80,pop_70,hsg_10,hsg_00,hsg_90,hsg_80,...,cp_id_fm_02,cp_id_acr_02,cp_fl_fm_02,cp_fl_acr_02,cp_sm_fm_02,cp_sm_acr_02,wd_fm_02,wd_acr_02,wd_p1_fm_02,wd_p1_acr_02
pct_rur_10,1.010638,-0.659563,-0.632413,-0.609024,-0.600122,-0.573532,-0.655588,-0.635564,-0.609952,-0.601341,...,-0.185445,-0.021443,-0.284107,-0.151847,0.007625,-0.089802,-0.064038,0.022319,-0.066614,-0.009814
pop_10,-0.659563,1.010638,1.007029,0.999402,0.992354,0.983846,1.008913,1.005261,0.997803,0.991379,...,-0.013602,-0.053696,0.113132,0.005704,0.079840,-0.008290,-0.025086,-0.094703,-0.012096,-0.053764
pop_00,-0.632413,1.007029,1.010638,1.008386,1.004385,0.999116,1.006896,1.008859,1.006237,1.003065,...,-0.011479,-0.046569,0.088084,-0.004890,0.084152,-0.010101,-0.038301,-0.107738,-0.029401,-0.073517
pop_90,-0.609024,0.999402,1.008386,1.010638,1.009471,1.006775,1.000903,1.007327,1.008809,1.008294,...,-0.010900,-0.041502,0.064140,-0.015339,0.089123,-0.011697,-0.052686,-0.118284,-0.046131,-0.090962
pop_80,-0.600122,0.992354,1.004385,1.009471,1.010638,1.009241,0.995328,1.004523,1.008443,1.009985,...,-0.006565,-0.038238,0.050664,-0.019872,0.091620,-0.011815,-0.058222,-0.124832,-0.055083,-0.102375
pop_70,-0.573532,0.983846,0.999116,1.006775,1.009241,1.010638,0.987604,0.999452,1.006238,1.008766,...,-0.005590,-0.028305,0.032524,-0.022101,0.100659,-0.009352,-0.063487,-0.129310,-0.063083,-0.105723
hsg_10,-0.655588,1.008913,1.006896,1.000903,0.995328,0.987604,1.010638,1.008141,1.001790,0.995859,...,-0.014123,-0.063414,0.099448,-0.005019,0.084568,-0.007325,-0.028133,-0.099372,-0.014281,-0.061723
hsg_00,-0.635564,1.005261,1.008859,1.007327,1.004523,0.999452,1.008141,1.010638,1.008481,1.005212,...,-0.010916,-0.057697,0.079487,-0.012999,0.091718,-0.007327,-0.038698,-0.110427,-0.028302,-0.078058
hsg_90,-0.609952,0.997803,1.006237,1.008809,1.008443,1.006238,1.001790,1.008481,1.010638,1.009309,...,-0.012874,-0.050069,0.055045,-0.021998,0.101172,-0.007814,-0.053816,-0.120889,-0.045991,-0.092872
hsg_80,-0.601341,0.991379,1.003065,1.008294,1.009985,1.008766,0.995859,1.005212,1.009309,1.010638,...,-0.004022,-0.041569,0.047552,-0.020676,0.099373,-0.010268,-0.056474,-0.125369,-0.052292,-0.102209


In [28]:
#Perform eigendecomposition on covariance matrix
eig_vals, eig_vecs = np.linalg.eig(cov_df)

In [30]:
eigvals_df=pd.DataFrame(eig_vals)
eigvals_df

,0
0,(70.1705281648+0j)
1,(24.8534806683+0j)
2,(21.8657776836+0j)
3,(8.31773982913+0j)
4,(7.5224389161+0j)
5,(6.3498051583+0j)
6,(3.74784221974+0j)
7,(2.90260840853+0j)
8,(2.47735564044+0j)
9,(2.39761293839+0j)


In [31]:
eigvecs_df=pd.DataFrame(eig_vecs)
eigvecs_df

,0,1,2,3,4,5,6,7,8,9,...,178,179,180,181,182,183,184,185,186,187
0,(0.0265166408309+0j),(-0.138636203398+0j),(-0.0856888241795+0j),(-0.0356870640313+0j),(-0.0553088725031+0j),(-0.00430042599827+0j),(-0.104310037579+0j),(0.0376571953585+0j),(0.0554641863231+0j),(0.0326329762538+0j),...,(0.0426049622303+0.016379141892j),(0.0601287753815-0.0337779325833j),(0.0601287753815+0.0337779325833j),(0.000498699205324+0j),(0.0132835177368+0j),(0.0635756813007+0j),(-0.0314871553222-0.0234298000658j),(-0.0314871553222+0.0234298000658j),(-0.00446678127175-0.0145359304693j),(-0.00446678127175+0.0145359304693j)
1,(-0.00459806255712+0j),(0.167579793098+0j),(0.0668379152236+0j),(0.0800206104921+0j),(-0.127333413807+0j),(0.0177761586216+0j),(0.014114854656+0j),(-0.0133934892393+0j),(0.0629233264485+0j),(0.0266703045673+0j),...,(-0.014703607679-0.0129205844372j),(-0.0011381397523+0.00350394656425j),(-0.0011381397523-0.00350394656425j),(-0.00952870268643+0j),(-0.00400125868397+0j),(-0.00772609307149+0j),(-0.000604077769393-0.00488640393544j),(-0.000604077769393+0.00488640393544j),(-0.00601932431904+0.000533141504357j),(-0.00601932431904-0.000533141504357j)
2,(-0.0022092437395+0j),(0.161160022825+0j),(0.0659584823748+0j),(0.0972946608008+0j),(-0.138021176769+0j),(0.015522164782+0j),(0.02511309736+0j),(-0.020834002238+0j),(0.0747222463578+0j),(0.0262646745501+0j),...,(-0.0181040275855-0.00297217151087j),(-0.00886262338177+0.00973230397738j),(-0.00886262338177-0.00973230397738j),(-0.0265343789063+0j),(-0.00595936372212+0j),(-0.0230006695353+0j),(0.00484160888404-0.00494889599666j),(0.00484160888404+0.00494889599666j),(-0.0102486155986-0.0118819404711j),(-0.0102486155986+0.0118819404711j)
3,(-1.85373259721e-05+0j),(0.155598765656+0j),(0.0653985597991+0j),(0.111391116349+0j),(-0.146074910722+0j),(0.0137218226156+0j),(0.0333701454886+0j),(-0.0256699083229+0j),(0.0790908049469+0j),(0.0227415770288+0j),...,(0.0213220182049-0.00866055325426j),(0.00369453217878+0.0248414307622j),(0.00369453217878-0.0248414307622j),(-0.00863473797543+0j),(-0.00494812108796+0j),(0.031218042656+0j),(0.0201468097025-0.00245200384814j),(0.0201468097025+0.00245200384814j),(-0.00361679623221-0.0139027022114j),(-0.00361679623221+0.0139027022114j)
4,(0.00112528848132+0j),(0.1522138298+0j),(0.0651656648638+0j),(0.121690953681+0j),(-0.149232867835+0j),(0.0110309484796+0j),(0.0394087477118+0j),(-0.0294230987494+0j),(0.0790303932664+0j),(0.0192630406612+0j),...,(0.0240236200808-0.0146079056016j),(0.0404100668381-0.0162473120763j),(0.0404100668381+0.0162473120763j),(0.00546413986947+0j),(0.0139635490636+0j),(-0.0236291809333+0j),(-0.052188072334-0.0270536582019j),(-0.052188072334+0.0270536582019j),(-0.0310226944591-0.000892341175645j),(-0.0310226944591+0.000892341175645j)
5,(0.00263997407593+0j),(0.147215690528+0j),(0.0656042780366+0j),(0.128064417876+0j),(-0.156636042924+0j),(0.0133186944491+0j),(0.0401968589562+0j),(-0.0266873709363+0j),(0.0801542917352+0j),(0.020364787074+0j),...,(0.0184157066891+0.038248971579j),(0.00886381020772-0.0381303859651j),(0.00886381020772+0.0381303859651j),(0.0161159685366+0j),(0.0117599004416+0j),(0.00522381405635+0j),(0.00716586048474-0.0290285210832j),(0.00716586048474+0.0290285210832j),(0.0502736257483-0.0263791273804j),(0.0502736257483+0.0263791273804j)
6,(-0.00358244743666+0j),(0.167393791181+0j),(0.0645722045125+0j),(0.0882736675552+0j),(-0.128530847393+0j),(0.0153710554859+0j),(0.0191785197494+0j),(-0.010073167987+0j),(0.054841640867+0j),(0.0235408948174+0j),...,(0.0274124019898+0.0141597053372j),(0.134935105381-0.0130031724661j),(0.134935105381+0.0130031724661j),(-0.0549121093989+0j),(-0.0200279685283+0j),(0.064785877548+0j),(-0.0870028374225-0.0604014292965j),(-0.0870028374225+0.0604014292965j),(-0.0664237317469-0.0315387466668j),(-0.0664237317469+0.0315387466668j)
7,(-0.00166976403791+0j),(0.162300174159+0j),(0.064102823536+0j),(0.103503704806+0j),(-0.137400016619+0j),(0.0134157562114+0j),(0.0264008727556+0j),(-0.0151127215428+0j),(0.0617285696231+

In [67]:
# Create a PCA model with 2 components: pca
pca = sklearnPCA(n_components=30)

# Fit the PCA instance to the scaled samples
pca.fit(scaler_df)

# Transform the scaled samples: pca_features
pca_features = pca.transform(scaler_df)

In [68]:
pcaft_df = pd.DataFrame(pca_features)
pcaft_df

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,-7.332806,2.383207,-1.462753,-2.901731,-2.156411,-0.604029,-1.751729,0.353591,-0.581663,1.093847,...,-0.583357,0.518737,-0.886625,-0.303208,-0.958589,-0.915894,1.317901,0.640484,-1.525782,1.362332
1,16.606917,-1.754641,-1.416658,1.815371,0.999392,5.888233,-3.897700,-0.581651,-0.131679,0.614855,...,0.040641,-1.870950,0.609291,1.869419,-1.076697,1.015641,-0.213553,0.450245,0.276415,0.448534
2,-4.443341,-2.766427,-0.575457,1.203539,2.399260,-2.792445,0.340048,1.409836,0.227039,-0.339867,...,-0.036099,-1.584812,-0.029423,-0.744830,-0.387433,-0.680654,-1.979382,1.705766,1.515943,-0.477948
3,-2.663562,-2.707438,-3.624859,0.660256,1.548063,1.555363,0.393621,-1.076042,-0.581093,-0.676190,...,0.510659,-0.736110,1.908947,0.301963,1.216803,0.380952,-1.631139,-0.059368,0.675722,-0.124325
4,3.629524,4.585776,-1.504190,-1.620919,-2.897930,-0.533025,-1.828342,2.304286,-1.173839,-0.109668,...,-0.814357,0.910773,-0.441205,1.987483,-0.462619,0.464343,-0.039309,-0.053999,1.082626,1.663789
5,0.006174,3.420939,-2.553215,-0.415745,-0.865347,0.612241,-1.988268,1.184277,0.146287,-0.647912,...,-1.505538,0.191844,-0.371117,-0.153747,-1.148985,0.750933,0.764812,-0.005762,0.886970,0.999242
6,-8.624536,1.662236,-2.081417,-0.736109,-2.735177,-1.433350,-1.034670,0.481807,1.187351,-0.018542,...,-0.857597,1.737904,-0.173684,-0.156522,0.544464,0.878046,-1.263245,0.272745,0.154906,-0.744610
7,0.836037,-1.248077,-4.083825,0.797929,0.480016,1.706316,0.501052,0.312246,0.234769,0.127224,...,1.546492,-1.282177,0.215876,-0.610721,1.959486,-0.353178,-0.242484,0.656767,-0.496322,1.086635
8,3.616707,-6.814513,5.789249,-2.094997,3.257664,-4.138341,2.303134,-0.454599,-1.616439,0.049135,...,-1.325630,1.807127,-1.635460,-0.922432,-0.706424,-0.233869,1.197274,0.358936,0.560555,-1.879076
9,-7.197445,1.736370,-3.013973,-2.326001,-1.198424,-1.197936,-1.077636,0.758463,-1.308875,0.507066,...,-0.606852,-0.561136,-1.730748,-0.351398,-0.078694,-0.038640,-0.658153,-1.086924,0.528646,0.424251


In [69]:
# Scree plot values: determin how many component explain over 90% of total variation.

vari= eigvals_df=pd.DataFrame(pca.explained_variance_ratio_)
vari['1']=''
vari.iloc[0,1]=vari.iloc[0,0]
for i in np.arange(1, vari.shape[0]):
    vari.iloc[i,1] = vari.iloc[i,0] + vari.iloc[i-1,1]
vari

,0,1
0,0.369319,0.369319
1,0.130808,0.500126
2,0.115083,0.615209
3,0.043778,0.658987
4,0.039592,0.698579
5,0.033420,0.731999
6,0.019725,0.751724
7,0.015277,0.767001
8,0.013039,0.78004
9,0.012619,0.792659
